In [116]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns 
import math
from sklearn.impute import KNNImputer

In [2]:
# Load the data
df = pd.read_csv("01_dataset.csv", index_col='customer_id')
df.head()

,label,SHORT_TERM_COUNT,MID_TERM_COUNT,LONG_TERM_COUNT,SHORT_TERM_COUNT_BANK,MID_TERM_COUNT_BANK,LONG_TERM_COUNT_BANK,SHORT_TERM_COUNT_NON_BANK,MID_TERM_COUNT_NON_BANK,LONG_TERM_COUNT_NON_BANK,...,ENQUIRIES_FROM_BANK_6M_9M,ENQUIRIES_FROM_BANK_9M_12M,ENQUIRIES_FROM_BANK_6M_12M,ENQUIRIES_FROM_BANK_3M_12M,ENQUIRIES_FROM_NON_BANK_3M_6M,ENQUIRIES_FROM_NON_BANK_6M_9M,ENQUIRIES_FROM_NON_BANK_9M_12M,ENQUIRIES_FROM_NON_BANK_6M_12M,ENQUIRIES_FROM_NON_BANK_3M_12M,OUTSTANDING_BAL_ALL_CURRENT
customer_id,,,,,,,,,,,,,,,,,,,,,
1639,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,2.000000,2.000000,2.000000,7.000000,3.666667,7.0,3.666667,3.666667,1000000.0
23717,0,10.0,1.0,NaN,7.0,1.0,1.0,4.0,NaN,NaN,...,2.000000,7.000000,2.000000,2.000000,7.000000,7.000000,7.0,7.000000,7.000000,NaN
19474,0,7.0,7.0,NaN,7.0,NaN,1.0,1.0,4.0,NaN,...,7.000000,5.571429,5.571429,5.571429,7.000000,7.000000,7.0,7.000000,7.000000,1000620.0
15505,0,7.0,1.0,1.0,NaN,1.0,NaN,7.0,1.0,1.0,...,6.090909,7.000000,NaN,NaN,4.500000,5.333333,7.0,5.333333,3.666667,1000170.0
1538,1,NaN,NaN,1.0,1.0,1.0,1.0,1.0,4.0,1.0,...,2.000000,2.000000,2.000000,2.000000,3.666667,7.000000,7.0,7.000000,3.666667,1000270.0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20000 entries, 1639 to 18772
Columns: 123 entries, label to OUTSTANDING_BAL_ALL_CURRENT
dtypes: float64(122), int64(1)
memory usage: 18.9 MB


In [4]:
df.describe()

,label,SHORT_TERM_COUNT,MID_TERM_COUNT,LONG_TERM_COUNT,SHORT_TERM_COUNT_BANK,MID_TERM_COUNT_BANK,LONG_TERM_COUNT_BANK,SHORT_TERM_COUNT_NON_BANK,MID_TERM_COUNT_NON_BANK,LONG_TERM_COUNT_NON_BANK,...,ENQUIRIES_FROM_BANK_6M_9M,ENQUIRIES_FROM_BANK_9M_12M,ENQUIRIES_FROM_BANK_6M_12M,ENQUIRIES_FROM_BANK_3M_12M,ENQUIRIES_FROM_NON_BANK_3M_6M,ENQUIRIES_FROM_NON_BANK_6M_9M,ENQUIRIES_FROM_NON_BANK_9M_12M,ENQUIRIES_FROM_NON_BANK_6M_12M,ENQUIRIES_FROM_NON_BANK_3M_12M,OUTSTANDING_BAL_ALL_CURRENT
count,20000.000000,18000.000000,18000.000000,18000.000000,18000.000000,18000.000000,18000.000000,18000.000000,18000.000000,18000.000000,...,18000.000000,18000.000000,18000.000000,18000.000000,18000.000000,18000.000000,18000.000000,18000.000000,18000.000000,1.800000e+04
mean,0.182200,6.040667,3.316833,1.063333,3.177000,1.610500,1.061333,3.850500,2.730833,1.002333,...,1.524688,1.956620,1.531301,1.013238,6.458641,6.591671,6.691829,6.402545,6.006104,1.000943e+06
std,0.386019,3.620554,3.147751,0.446094,2.211548,1.547231,0.437320,2.550394,2.519339,0.089415,...,4.182101,4.304385,3.967109,3.560061,1.147119,1.022613,0.914260,1.177294,1.431080,4.007350e+03
min,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,-3.000000,-3.000000,-3.000000,-3.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1.000000e+06
25%,0.000000,4.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,-3.000000,-3.000000,-3.000000,-3.000000,7.000000,7.000000,7.000000,7.000000,4.500000,1.000100e+06
50%,0.000000,7.000000,1.000000,1.000000,4.000000,1.000000,1.000000,4.000000,1.000000,1.000000,...,2.000000,2.000000,2.000000,2.000000,7.000000,7.000000,7.000000,7.000000,7.000000,1.000290e+06
75%,0.000000,7.000000,4.000000,1.000000,4.000000,1.000000,1.000000,4.000000,4.000000,1.000000,...,7.000000,7.000000,5.333333,3.000000,7.000000,7.000000,7.000000,7.000000,7.000000,1.000770e+06
max,1.000000,25.000000,25.000000,7.000000,13.000000,22.000000,7.000000,16.000000,19.000000,7.000000,...,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,1.169190e+06


In [5]:
# Check for missing values, find columns with missing values
null_count = df.isnull().sum()
null_cols = null_count[null_count>0].index.to_list()
print("The number of columns with missing values:", len(null_cols))

The number of columns with missing values: 122


=> Tất cả các cột ngoại trừ customer_id và label đều có giá trị null

In [6]:
df.notnull().all(axis=1).sum()

0

=> Tất cả các hàng đều có ít nhất một giá trị null => Không thể xóa hết các hàng có chứa giá trị rỗng được, có thể nghĩ đến hướng impute data

In [7]:
cols = df.columns
integer_cols = [col for col in cols if 'enquiries' in col.lower() or "count" in col.lower() or "number" in col.lower() or 'num' in col.lower() or 'month' in col.lower()]
integer_cols[:5]

['SHORT_TERM_COUNT',
 'MID_TERM_COUNT',
 'LONG_TERM_COUNT',
 'SHORT_TERM_COUNT_BANK',
 'MID_TERM_COUNT_BANK']

In [8]:
# Kiểm tra xem số lượt tra cứu có hợp lệ không (phải là số nguyên)

def check_invalid_enquiries(value):
    if pd.isna(value):
        return False
    else:
        if int(value) != value:
            return False 
        return True
    
# Tìm các index tương ứng với các dòng có số lượt tra cứu không hợp lệ

invalid_rows = []
for col in integer_cols:
    temp = df[col].apply(check_invalid_enquiries)
    invalid_rows.append(temp[temp == False].index.to_list())

In [9]:
df.loc[invalid_rows[-1]][integer_cols[-1]]

customer_id
1639     3.666667
15505    3.666667
1538     3.666667
20339    3.250000
6629     5.333333
           ...   
1620     4.500000
2022     4.500000
9301     4.500000
22422    4.500000
11823    5.333333
Name: ENQUIRIES_FROM_NON_BANK_3M_12M, Length: 7575, dtype: float64

In [10]:
def check_not_int(value):
    if pd.isna(value):
        return True
    return int(value) == value

for col in df.columns:
    if df[col].apply(check_not_int).sum() != df.shape[0]:
        print(col, df[col].apply(check_not_int).sum())

OUTSTANDING_BAL_LOAN_3M 9341
OUTSTANDING_BAL_LOAN_6M 9129
OUTSTANDING_BAL_LOAN_9M 5133
OUTSTANDING_BAL_LOAN_12M 6073
OUTSTANDING_BAL_CC_3M 13670
OUTSTANDING_BAL_CC_6M 13210
OUTSTANDING_BAL_CC_9M 10688
OUTSTANDING_BAL_CC_12M 11187
OUTSTANDING_BAL_ALL_3M 8993
OUTSTANDING_BAL_ALL_6M 8767
OUTSTANDING_BAL_ALL_9M 4818
OUTSTANDING_BAL_ALL_12M 5821
OUTSTANDING_BAL_LOAN_3M_6M 6474
OUTSTANDING_BAL_LOAN_6M_9M 7723
OUTSTANDING_BAL_LOAN_9M_12M 3289
OUTSTANDING_BAL_LOAN_6M_12M 7301
OUTSTANDING_BAL_LOAN_3M_12M 5569
OUTSTANDING_BAL_CC_3M_6M 12551
OUTSTANDING_BAL_CC_6M_9M 13122
OUTSTANDING_BAL_CC_9M_12M 9560
OUTSTANDING_BAL_CC_6M_12M 13060
OUTSTANDING_BAL_CC_3M_12M 11928
OUTSTANDING_BAL_ALL_3M_6M 5385
OUTSTANDING_BAL_ALL_6M_9M 6540
OUTSTANDING_BAL_ALL_9M_12M 3010
OUTSTANDING_BAL_ALL_6M_12M 6327
OUTSTANDING_BAL_ALL_3M_12M 4809
ENQUIRIES_3M_6M 15085
ENQUIRIES_6M_9M 15816
ENQUIRIES_9M_12M 16342
ENQUIRIES_6M_12M 14134
ENQUIRIES_3M_12M 12057
ENQUIRIES_FROM_BANK_3M_6M 18522
ENQUIRIES_FROM_BANK_6M_9M 18144
EN

In [11]:
# Xử lý các giá trị không hợp lệ bằng cách làm tròn giá trị về số nguyên gần nhất

def replace_float_to_int(value):
    if pd.isna(value):
        return value
    return math.floor(value) if math.ceil(value) - value > value - math.floor(value) else math.ceil(value)

for col in integer_cols:
    df[col] = df[col].apply(replace_float_to_int)

In [12]:
df.loc[invalid_rows[-1]][integer_cols[-1]]

customer_id
1639     4.0
15505    4.0
1538     4.0
20339    3.0
6629     5.0
        ... 
1620     5.0
2022     5.0
9301     5.0
22422    5.0
11823    5.0
Name: ENQUIRIES_FROM_NON_BANK_3M_12M, Length: 7575, dtype: float64

In [13]:
df.columns

Index(['label', 'SHORT_TERM_COUNT', 'MID_TERM_COUNT', 'LONG_TERM_COUNT',
       'SHORT_TERM_COUNT_BANK', 'MID_TERM_COUNT_BANK', 'LONG_TERM_COUNT_BANK',
       'SHORT_TERM_COUNT_NON_BANK', 'MID_TERM_COUNT_NON_BANK',
       'LONG_TERM_COUNT_NON_BANK',
       ...
       'ENQUIRIES_FROM_BANK_6M_9M', 'ENQUIRIES_FROM_BANK_9M_12M',
       'ENQUIRIES_FROM_BANK_6M_12M', 'ENQUIRIES_FROM_BANK_3M_12M',
       'ENQUIRIES_FROM_NON_BANK_3M_6M', 'ENQUIRIES_FROM_NON_BANK_6M_9M',
       'ENQUIRIES_FROM_NON_BANK_9M_12M', 'ENQUIRIES_FROM_NON_BANK_6M_12M',
       'ENQUIRIES_FROM_NON_BANK_3M_12M', 'OUTSTANDING_BAL_ALL_CURRENT'],
      dtype='object', length=123)

In [14]:
df['NUM_NEW_LOAN_TAKEN_3M'].apply(check_invalid_enquiries).sum()

18000

In [15]:
df[['OUTSTANDING_BAL_ALL_3M_6M', 'OUTSTANDING_BAL_ALL_3M', 'OUTSTANDING_BAL_ALL_6M']].head()

,OUTSTANDING_BAL_ALL_3M_6M,OUTSTANDING_BAL_ALL_3M,OUTSTANDING_BAL_ALL_6M
customer_id,,,
1639,1.000000e+06,1.000000e+06,1.000000e+06
23717,1.000014e+06,1.000433e+06,1.000320e+06
19474,1.000012e+06,1.000607e+06,1.000522e+06
15505,1.000010e+06,1.000097e+06,1.000098e+06
1538,1.000013e+06,NaN,1.000230e+06


In [16]:
# Kiểm tra lại xem các cột số nguyên đã hợp lệ chưa
for col in df.columns:
    if df[col].apply(check_not_int).sum() != df.shape[0]:
        print(col, df[col].apply(check_not_int).sum())

OUTSTANDING_BAL_LOAN_3M 9341
OUTSTANDING_BAL_LOAN_6M 9129
OUTSTANDING_BAL_LOAN_9M 5133
OUTSTANDING_BAL_LOAN_12M 6073
OUTSTANDING_BAL_CC_3M 13670
OUTSTANDING_BAL_CC_6M 13210
OUTSTANDING_BAL_CC_9M 10688
OUTSTANDING_BAL_CC_12M 11187
OUTSTANDING_BAL_ALL_3M 8993
OUTSTANDING_BAL_ALL_6M 8767
OUTSTANDING_BAL_ALL_9M 4818
OUTSTANDING_BAL_ALL_12M 5821
OUTSTANDING_BAL_LOAN_3M_6M 6474
OUTSTANDING_BAL_LOAN_6M_9M 7723
OUTSTANDING_BAL_LOAN_9M_12M 3289
OUTSTANDING_BAL_LOAN_6M_12M 7301
OUTSTANDING_BAL_LOAN_3M_12M 5569
OUTSTANDING_BAL_CC_3M_6M 12551
OUTSTANDING_BAL_CC_6M_9M 13122
OUTSTANDING_BAL_CC_9M_12M 9560
OUTSTANDING_BAL_CC_6M_12M 13060
OUTSTANDING_BAL_CC_3M_12M 11928
OUTSTANDING_BAL_ALL_3M_6M 5385
OUTSTANDING_BAL_ALL_6M_9M 6540
OUTSTANDING_BAL_ALL_9M_12M 3010
OUTSTANDING_BAL_ALL_6M_12M 6327
OUTSTANDING_BAL_ALL_3M_12M 4809


In [17]:
df.columns[100:]

Index(['ENQUIRIES_FROM_NON_BANK_12M', 'ENQUIRIES_FOR_LOAN_12M',
       'ENQUIRIES_FOR_CC_12M', 'ENQUIRIES_FROM_BANK_FOR_LOAN_12M',
       'ENQUIRIES_FROM_NON_BANK_FOR_LOAN_12M',
       'ENQUIRIES_FROM_BANK_FOR_CC_12M', 'ENQUIRIES_FROM_NON_BANK_FOR_CC_12M',
       'ENQUIRIES_3M_6M', 'ENQUIRIES_6M_9M', 'ENQUIRIES_9M_12M',
       'ENQUIRIES_6M_12M', 'ENQUIRIES_3M_12M', 'ENQUIRIES_FROM_BANK_3M_6M',
       'ENQUIRIES_FROM_BANK_6M_9M', 'ENQUIRIES_FROM_BANK_9M_12M',
       'ENQUIRIES_FROM_BANK_6M_12M', 'ENQUIRIES_FROM_BANK_3M_12M',
       'ENQUIRIES_FROM_NON_BANK_3M_6M', 'ENQUIRIES_FROM_NON_BANK_6M_9M',
       'ENQUIRIES_FROM_NON_BANK_9M_12M', 'ENQUIRIES_FROM_NON_BANK_6M_12M',
       'ENQUIRIES_FROM_NON_BANK_3M_12M', 'OUTSTANDING_BAL_ALL_CURRENT'],
      dtype='object')

In [18]:
# Tìm những cột giống hệt nhau (không kể đến giá trị null)

drop_list = set()

correlation = df.corr()
for col in correlation.columns:
    for row in correlation.index:
        if correlation.loc[row, col] == 1 and row != col:
            drop_list.add(col)
            drop_list.add(row)

drop_list = list(drop_list)
drop_list

['NUM_NEW_LOAN_TAKEN_NON_BANK_9M',
 'NUM_NEW_LOAN_TAKEN_BANK_6M',
 'NUM_NEW_LOAN_TAKEN_3M',
 'NUM_NEW_LOAN_TAKEN_NON_BANK_12M',
 'NUM_NEW_LOAN_TAKEN_BANK_3M',
 'NUM_NEW_LOAN_TAKEN_NON_BANK_3M',
 'NUM_NEW_LOAN_TAKEN_BANK_9M',
 'NUM_NEW_LOAN_TAKEN_6M',
 'NUM_NEW_LOAN_TAKEN_NON_BANK_6M',
 'NUM_NEW_LOAN_TAKEN_BANK_12M',
 'NUM_NEW_LOAN_TAKEN_12M',
 'NUM_NEW_LOAN_TAKEN_9M']

In [19]:
# Drop những cột có giá trị giống hệt nhau (không kể đến những giá trị null)
df.drop(columns=['NUM_NEW_LOAN_TAKEN_BANK_3M', 'NUM_NEW_LOAN_TAKEN_NON_BANK_3M', 'NUM_NEW_LOAN_TAKEN_BANK_6M', 'NUM_NEW_LOAN_TAKEN_NON_BANK_6M', 'NUM_NEW_LOAN_TAKEN_BANK_9M', 'NUM_NEW_LOAN_TAKEN_NON_BANK_9M', 'NUM_NEW_LOAN_TAKEN_BANK_12M', 'NUM_NEW_LOAN_TAKEN_NON_BANK_12M'], inplace=True)

In [24]:
correlation = df.corr()
for col in correlation.columns:
    for row in correlation.index:
        if correlation.loc[row, col] > 0.8 and row != col:
            print('{}\t{}\t{}'.format(row, col, correlation.loc[row, col]))

MID_TERM_COUNT_NON_BANK	MID_TERM_COUNT	0.882600415611006
LONG_TERM_COUNT_BANK	LONG_TERM_COUNT	0.9764251785888444
NUMBER_OF_LOANS_BANK	SHORT_TERM_COUNT_BANK	0.8120184889502003
LONG_TERM_COUNT	LONG_TERM_COUNT_BANK	0.9764251785888444
MID_TERM_COUNT	MID_TERM_COUNT_NON_BANK	0.882600415611006
NUM_NEW_LOAN_TAKEN_12M	NUMBER_OF_LOANS	0.8090259280713363
SHORT_TERM_COUNT_BANK	NUMBER_OF_LOANS_BANK	0.8120184889502003
NUMBER_OF_RELATIONSHIP_NON_BANK	NUMBER_OF_LOANS_NON_BANK	0.8859962989835735
NUMBER_OF_CREDIT_CARDS_BANK	NUMBER_OF_CREDIT_CARDS	0.9280865598845992
NUMBER_OF_RELATIONSHIP	NUMBER_OF_CREDIT_CARDS	0.8389219308270561
NUMBER_OF_RELATIONSHIP_BANK	NUMBER_OF_CREDIT_CARDS	0.8535306983503078
NUMBER_OF_CREDIT_CARDS	NUMBER_OF_CREDIT_CARDS_BANK	0.9280865598845992
NUMBER_OF_RELATIONSHIP_BANK	NUMBER_OF_CREDIT_CARDS_BANK	0.910122102950203
NUMBER_OF_CREDIT_CARDS	NUMBER_OF_RELATIONSHIP	0.8389219308270561
NUMBER_OF_RELATIONSHIP_BANK	NUMBER_OF_RELATIONSHIP	0.8609565470667894
NUMBER_OF_CREDIT_CARDS	NUMBER_OF

In [29]:
df['SHORT_TERM_COUNT_BANK']+df['MID_TERM_COUNT_BANK']+df['LONG_TERM_COUNT_BANK']

customer_id
1639      3.0
23717     9.0
19474     NaN
15505     NaN
1538      3.0
         ... 
13215     6.0
938       3.0
11823     6.0
14609    12.0
18772     3.0
Length: 20000, dtype: float64

In [32]:
df['NUMBER_OF_LOANS_BANK']+df['NUMBER_OF_LOANS_NON_BANK']

customer_id
1639      2.0
23717    11.0
19474    14.0
15505     8.0
1538      NaN
         ... 
13215     5.0
938       5.0
11823     8.0
14609    14.0
18772     5.0
Length: 20000, dtype: float64

In [31]:
df['NUMBER_OF_LOANS']

customer_id
1639      1.0
23717     NaN
19474    13.0
15505     7.0
1538      NaN
         ... 
13215     4.0
938       4.0
11823     7.0
14609    13.0
18772     4.0
Name: NUMBER_OF_LOANS, Length: 20000, dtype: float64

In [34]:
df.columns[:20]

Index(['label', 'SHORT_TERM_COUNT', 'MID_TERM_COUNT', 'LONG_TERM_COUNT',
       'SHORT_TERM_COUNT_BANK', 'MID_TERM_COUNT_BANK', 'LONG_TERM_COUNT_BANK',
       'SHORT_TERM_COUNT_NON_BANK', 'MID_TERM_COUNT_NON_BANK',
       'LONG_TERM_COUNT_NON_BANK', 'NUMBER_OF_LOANS', 'NUMBER_OF_LOANS_BANK',
       'NUMBER_OF_LOANS_NON_BANK', 'NUMBER_OF_CREDIT_CARDS',
       'NUMBER_OF_CREDIT_CARDS_BANK', 'NUMBER_OF_CREDIT_CARDS_NON_BANK',
       'NUMBER_OF_RELATIONSHIP', 'NUMBER_OF_RELATIONSHIP_BANK',
       'NUMBER_OF_RELATIONSHIP_NON_BANK', 'NUM_NEW_LOAN_TAKEN_3M'],
      dtype='object')

In [81]:
# Kiểm tra xem tổng các chỉ số của ngân hàng và phi ngân hàng có bằng chỉ số tổng quát không

bank_cols = []
non_bank_cols = []
total_cols = []

for col in df.columns:
    if col == 'label':
        continue
    if 'NON_BANK' in col:
        non_bank_cols.append(col)
    elif 'BANK' in col:
        bank_cols.append(col)
    else:
        total_cols.append(col)

for i in range (len(bank_cols)):
    if 'ENQUIRIES' in bank_cols[i]:
        if df[bank_cols[i]].sum() + df[non_bank_cols[i]].sum() == df[bank_cols[i].replace('_FROM_BANK', '')].sum():
            print(bank_cols[i].replace('_FROM_BANK', ''))
    elif df[bank_cols[i]].sum() + df[non_bank_cols[i]].sum() == df[bank_cols[i].replace('_BANK', '')].sum():
        print(bank_cols[i].replace('_BANK', ''))

=> Tổng các chỉ số của ngân hàng và phi ngân hàng không bằng chỉ số tổng quát

In [115]:
# Kiểm tra xem tổng các chỉ số của thẻ tín dụng và các khoản vay có bằng các chỉ số của tất cả các sản phẩm tài chính không

loan_cols = []
all_cols = []
cc_cols = []

for col in df.columns:
    if 'ALL' in col:
        all_cols.append(col)
    elif 'CC' in col:
        cc_cols.append(col)

for i in range (len(all_cols)):
    if df[all_cols[i]].sum() == df[cc_cols[i]].sum()+df[all_cols[i].replace('ALL', 'LOAN')].sum():
        print(all_cols[i])

=> Tổng các chỉ số của thẻ tín dụng và các khoản vay không bằng các chỉ số của tất cả các sản phẩm tài chính

In [85]:
i = 0
df[all_cols[i]]

customer_id
1639     1.000000e+06
23717    1.000433e+06
19474    1.000607e+06
15505    1.000097e+06
1538              NaN
             ...     
13215    1.000033e+06
938      1.000023e+06
11823    1.001173e+06
14609             NaN
18772    1.000123e+06
Name: OUTSTANDING_BAL_ALL_3M, Length: 20000, dtype: float64

In [106]:
df.corr()['label'].sort_values(ascending=True)[:20]

OUTSTANDING_BAL_LOAN_9M_12M       -0.409176
OUTSTANDING_BAL_ALL_9M_12M        -0.365516
OUTSTANDING_BAL_LOAN_6M_9M        -0.364709
NUMBER_OF_LOANS                   -0.350134
NUMBER_OF_RELATIONSHIP            -0.332823
NUM_NEW_LOAN_TAKEN_12M            -0.326216
OUTSTANDING_BAL_ALL_6M_9M         -0.321813
SHORT_TERM_COUNT                  -0.312538
NUM_NEW_LOAN_TAKEN_9M             -0.304447
OUTSTANDING_BAL_LOAN_6M_12M       -0.280451
OUTSTANDING_BAL_LOAN_3M_6M        -0.272169
NUMBER_OF_LOANS_NON_BANK          -0.264508
NUMBER_OF_RELATIONSHIP_BANK       -0.260890
NUMBER_OF_LOANS_BANK              -0.259070
SHORT_TERM_COUNT_BANK             -0.258823
NUM_NEW_LOAN_TAKEN_6M             -0.255318
OUTSTANDING_BAL_ALL_6M_12M        -0.248647
NUMBER_OF_RELATIONSHIP_NON_BANK   -0.246642
SHORT_TERM_COUNT_NON_BANK         -0.219785
OUTSTANDING_BAL_ALL_3M_6M         -0.216759
Name: label, dtype: float64

In [119]:
imputer = KNNImputer(n_neighbors=5)
df_imputed = imputer.fit_transform(df)

In [123]:
features = imputer.get_feature_names_out()
df_imputed = pd.DataFrame(df_imputed, columns=features, index=df.index)

In [126]:
df_imputed.describe()

,label,SHORT_TERM_COUNT,MID_TERM_COUNT,LONG_TERM_COUNT,SHORT_TERM_COUNT_BANK,MID_TERM_COUNT_BANK,LONG_TERM_COUNT_BANK,SHORT_TERM_COUNT_NON_BANK,MID_TERM_COUNT_NON_BANK,LONG_TERM_COUNT_NON_BANK,...,ENQUIRIES_FROM_BANK_6M_9M,ENQUIRIES_FROM_BANK_9M_12M,ENQUIRIES_FROM_BANK_6M_12M,ENQUIRIES_FROM_BANK_3M_12M,ENQUIRIES_FROM_NON_BANK_3M_6M,ENQUIRIES_FROM_NON_BANK_6M_9M,ENQUIRIES_FROM_NON_BANK_9M_12M,ENQUIRIES_FROM_NON_BANK_6M_12M,ENQUIRIES_FROM_NON_BANK_3M_12M,OUTSTANDING_BAL_ALL_CURRENT
count,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,...,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,2.000000e+04
mean,0.182200,6.038200,3.328090,1.062340,3.177370,1.605580,1.061980,3.851350,2.726890,1.002220,...,1.479860,1.937890,1.511120,1.006900,6.492070,6.611670,6.706260,6.434730,6.067580,1.000921e+06
std,0.386019,3.505428,3.102785,0.431512,2.134289,1.501694,0.426487,2.464387,2.466544,0.085249,...,4.078347,4.195322,3.875373,3.476856,1.062673,0.961713,0.868098,1.102233,1.332242,3.867965e+03
min,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,-3.000000,-3.000000,-3.000000,-3.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1.000000e+06
25%,0.000000,4.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,-3.000000,-3.000000,-3.000000,-3.000000,7.000000,7.000000,7.000000,6.200000,5.000000,1.000100e+06
50%,0.000000,7.000000,1.000000,1.000000,4.000000,1.000000,1.000000,4.000000,1.000000,1.000000,...,2.000000,2.000000,2.000000,2.000000,7.000000,7.000000,7.000000,7.000000,7.000000,1.000290e+06
75%,0.000000,7.000000,4.000000,1.000000,4.000000,1.000000,1.000000,4.000000,4.000000,1.000000,...,6.000000,7.000000,5.000000,3.000000,7.000000,7.000000,7.000000,7.000000,7.000000,1.000760e+06
max,1.000000,25.000000,25.000000,7.000000,13.000000,22.000000,7.000000,16.000000,19.000000,7.000000,...,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,1.169190e+06


In [127]:
df.describe()

,label,SHORT_TERM_COUNT,MID_TERM_COUNT,LONG_TERM_COUNT,SHORT_TERM_COUNT_BANK,MID_TERM_COUNT_BANK,LONG_TERM_COUNT_BANK,SHORT_TERM_COUNT_NON_BANK,MID_TERM_COUNT_NON_BANK,LONG_TERM_COUNT_NON_BANK,...,ENQUIRIES_FROM_BANK_6M_9M,ENQUIRIES_FROM_BANK_9M_12M,ENQUIRIES_FROM_BANK_6M_12M,ENQUIRIES_FROM_BANK_3M_12M,ENQUIRIES_FROM_NON_BANK_3M_6M,ENQUIRIES_FROM_NON_BANK_6M_9M,ENQUIRIES_FROM_NON_BANK_9M_12M,ENQUIRIES_FROM_NON_BANK_6M_12M,ENQUIRIES_FROM_NON_BANK_3M_12M,OUTSTANDING_BAL_ALL_CURRENT
count,20000.000000,18000.000000,18000.000000,18000.000000,18000.000000,18000.000000,18000.000000,18000.000000,18000.000000,18000.000000,...,18000.000000,18000.000000,18000.000000,18000.000000,18000.000000,18000.000000,18000.000000,18000.000000,18000.000000,1.800000e+04
mean,0.182200,6.040667,3.316833,1.063333,3.177000,1.610500,1.061333,3.850500,2.730833,1.002333,...,1.534389,1.964556,1.544389,1.027111,6.490056,6.612056,6.703722,6.432167,6.062278,1.000943e+06
std,0.386019,3.620554,3.147751,0.446094,2.211548,1.547231,0.437320,2.550394,2.519339,0.089415,...,4.189380,4.309975,3.977145,3.573526,1.090805,0.981183,0.886495,1.128924,1.367785,4.007350e+03
min,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,-3.000000,-3.000000,-3.000000,-3.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1.000000e+06
25%,0.000000,4.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,-3.000000,-3.000000,-3.000000,-3.000000,7.000000,7.000000,7.000000,7.000000,5.000000,1.000100e+06
50%,0.000000,7.000000,1.000000,1.000000,4.000000,1.000000,1.000000,4.000000,1.000000,1.000000,...,2.000000,2.000000,2.000000,2.000000,7.000000,7.000000,7.000000,7.000000,7.000000,1.000290e+06
75%,0.000000,7.000000,4.000000,1.000000,4.000000,1.000000,1.000000,4.000000,4.000000,1.000000,...,7.000000,7.000000,5.000000,3.000000,7.000000,7.000000,7.000000,7.000000,7.000000,1.000770e+06
max,1.000000,25.000000,25.000000,7.000000,13.000000,22.000000,7.000000,16.000000,19.000000,7.000000,...,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,1.169190e+06


In [151]:
X = df_imputed.drop(columns='label')
y = df_imputed['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test)

param = {'num_leaves': 50, 'objective': 'binary'}
param['metric'] = 'auc'

num_round = 50
bst = lgb.train(param, train_data, num_round, valid_sets=[test_data])

[LightGBM] [Info] Number of positive: 2908, number of negative: 13092
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.011142 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 10212
[LightGBM] [Info] Number of data points in the train set: 16000, number of used features: 110
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.181750 -> initscore=-1.504536
[LightGBM] [Info] Start training from score -1.504536


In [152]:
ybar = bst.predict(X_train)
ybar = [1 if x >= 0.5 else 0 for x in ybar]
ypred = bst.predict(X_test)
ypred = [1 if x >= 0.5 else 0 for x in ypred]
print('Accuracy for train data: {}%'.format(accuracy_score(y_train, ybar)*100))
print('Accuracy for test data: {}%'.format(accuracy_score(y_test, ypred)*100))

Accuracy for train data: 90.975%
Accuracy for test data: 88.8%
